In [ ]:
# Script is a test script connecting environment to google skd
    # works on a dummy image

In [1]:
import os
import vertexai
from vertexai.generative_models import GenerativeModel, Part
from PIL import Image

In [2]:
# --- Configuration ---
# Set your Google Cloud project details here.
GCP_PROJECT_ID = "mlexpimgsorting-v2"  # Your new, working Project ID
GCP_LOCATION = "us-central1"

# This sets the specific environment variable the Google SDK requires
os.environ['GOOGLE_APPLICATION_CREDENTIALS'] = "/Users/natalyagrokh/AI/img_curation/mlexpimgsorting-v2-c5a570b110c3.json"

In [3]:
# Uses the Gemini 1.5 Pro model on Vertex AI to perform image inpainting.

def perform_inpainting(image_path: str, mask_path: str, prompt: str):
    
    print(f"Starting inpainting process for: {image_path}")

    try:
        # 1. Initialize the Vertex AI SDK with your project and location.
        vertexai.init(project=GCP_PROJECT_ID, location=GCP_LOCATION)

        # 2. Load the specific Gemini model from the initialized SDK.
        # Note: The stable model name is "gemini-1.5-pro-001".
        model = GenerativeModel("gemini-2.5-pro")

        # 3. Prepare the image and mask for the API call.
        print("Loading image and mask files...")
        with open(image_path, "rb") as f:
            base_image_bytes = f.read()
        
        with open(mask_path, "rb") as f:
            mask_image_bytes = f.read()
        
        base_image_part = Part.from_data(data=base_image_bytes, mime_type="image/png")
        mask_image_part = Part.from_data(data=mask_image_bytes, mime_type="image/png")
        
        # 4. Construct the multimodal prompt using the correctly formatted parts.
        contents = [prompt, base_image_part, mask_image_part]

        # 5. Make the API call using the model object.
        print("Sending request to Vertex AI API...")
        response = model.generate_content(contents)

        print("Successfully received response from the model.")
        
        # 6. Extract and save the generated image from the response.
        0utput_image_bytes = response.candidates[0].content.parts[0].inline_data.data
        with open("output_image.png", "wb") as f:
            f.write(output_image_bytes)
        print("Saved generated image to output_image.png")

        return response

    except Exception as e:
        print(f"\n--- An Error Occurred ---")
        print(f"Error Type: {type(e).__name__}")
        print(f"Error Details: {e}")
        return None

In [4]:
# --- MAIN EXECUTION BLOCK ---
if __name__ == "__main__":
    # --- Example Usage ---
    # For inpainting, you need three things: the base image, a mask image,
    # and a text prompt.
    example_image_path = "face_image.png"
    example_mask_path = "mask.png" # You will need to create this mask file.
    example_prompt = "This is an image of a person's face with a mask. Inpaint the masked area to remove the glasses, keeping the expression neutral."

    # Create dummy files if they don't exist for demonstration.
    if not os.path.exists(example_image_path):
        print(f"Creating a dummy image file at '{example_image_path}'.")
        dummy_image = Image.new('RGB', (100, 100), color='red')
        dummy_image.save(example_image_path)

    if not os.path.exists(example_mask_path):
        print(f"Creating a dummy mask file at '{example_mask_path}'.")
        # In a real scenario, the white area of the mask would cover the glasses.
        dummy_mask = Image.new('RGB', (100, 100), color='black')
        # For this dummy file, we'll make a white square in the middle.
        for x in range(25, 75):
            for y in range(25, 75):
                dummy_mask.putpixel((x, y), (255, 255, 255))
        dummy_mask.save(example_mask_path)

    api_response = perform_inpainting(example_image_path, example_mask_path, example_prompt)

    if api_response:
        print("\n--- Process Complete ---")

Starting inpainting process for: face_image.png
Loading image and mask files...
Sending request to Vertex AI API...


/Users/natalyagrokh/miniconda3/envs/img_curation_v3/lib/python3.10/site-packages/vertexai/generative_models/_generative_models.py:433: UserWarning: This feature is deprecated as of June 24, 2025 and will be removed on June 24, 2026. For details, see https://cloud.google.com/vertex-ai/generative-ai/docs/deprecations/genai-vertexai-sdk.
  warning_logs.show_deprecation_warning()
E0000 00:00:1759493449.967962 1194559 alts_credentials.cc:93] ALTS creds ignored. Not running on GCP and untrusted ALTS is not enabled.


Successfully received response from the model.
Saved generated image to output_image.png

--- Process Complete ---
